In [10]:
import os
import tweepy as tw
import pandas as pd
import re
import glob
from datetime import datetime

%run ./fair_keys.ipynb

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [11]:
today = datetime.today()
#filename = str("tweet_data_"+str(today.day)+str(today.month)+str(today.year)+".csv")
filename = 'tweet_data_1932022.csv'
df = pd.read_csv(filename)

In [25]:
def savedata(keys):
    def get_related_tweets(key_word):
        tweet_keyword = []
        twitter_users = []
        #twitter_users_id = []
        tweet_time = []
        tweet_string = [] 
        tweet_countRT = []
        tweet_fav = []
        for tweet in tw.Cursor(api.search_tweets,q=key_word,tweet_mode="extended",include_entities=True).items(40):
                if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                    if tweet.lang == "en":
                        twitter_users.append(tweet.user.screen_name)
                        #twitter_users_id.append(tweet.user.id)
                        tweet_time.append(tweet.created_at)
                        tweet_string.append(tweet.full_text)
                        tweet_countRT.append(tweet.retweet_count)
                        tweet_fav.append(tweet.favorite_count)
                        tweet_keyword.append(key_word)
        df = pd.DataFrame({'Keyword':tweet_keyword,'User':twitter_users,'Retweet':tweet_countRT,'Likes':tweet_fav, 'Tweet': tweet_string, 'Time': tweet_time})
        #df.to_csv(f"{key_word}.csv")
        return df

    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    filename = 'tweet_data_1932022.csv'
    df = pd.read_csv(filename)
    os.remove(filename)

    for keyword in keys:
        df = pd.concat([df,get_related_tweets(keyword)])

    df.drop_duplicates(keep='last',inplace=True)
    df.sort_values(by=['Keyword'],inplace=True)
    df.to_csv(filename,encoding='utf-8',index=False)
    print('save complete',current_time)
    return df

In [33]:
keys = df['Keyword'].tolist()
keys = list(set(keys))
print(keys)
keyword = str(input('Enter Keyword: ')).lower()
print('Keyword:',keyword)
if keyword not in keys:
    print(f'{keyword} not in Database. Do you want to search?')
    Ans = str(input()).lower()
    print(Ans)
    if Ans == 'yes':
        keys.append(keyword)
        df = savedata(keys)
        print(df.loc[df['Keyword']==keyword,['Tweet']])
    else:
        pass
    
else:
    print(df.loc[df['Keyword']==keyword,['Tweet']])

['sport anime', 'bl anime', 'seinen', 'harem', 'mappa', 'from manga', 'anime comedy', 'animation studio', 'anime romance', 'school life', 'anime', 'shounen', 'shoujo', 'from novel', 'shounen ai', 'animation', 'pixar', 'fantasy anime', 'slice of life anime']
Keyword: pixar
                                                  Tweet
0     @nas_no1r Soul, Spider-Man, princes and frog, ...
1          Love it so much  🥹❤️ https://t.co/F7TnCxSk6I
2     @Superherofparis @dovecharts @Pixar @notladybu...
3     Coco is one of Pixar’s best films. https://t.c...
4     @rynluvszeha pixar jimin vs. walmart jimin… pi...
...                                                 ...
1454  @newsfrmhome are you trying to say anime is re...
1455  if i could watch a pixar movie for the first t...
1456  how to market a movie in 2022 😑\n\nhttps://t.c...
1457  @cosmicfizzypop To be fair it wouldn't be the ...
1458  Still weird to me that Pixar is making a sci-f...

[95 rows x 1 columns]


In [28]:
print(keys)

['animation studio', 'anime romance', 'school life', 'shoujo', 'shounen ai', 'animation', 'fantasy anime', 'anime', 'sport anime', 'bl anime', 'from manga', 'anime comedy', 'pixar', 'from novel', 'shounen', 'seinen', 'harem', 'slice of life anime', 'mappa']
